In [1]:
import os
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from pymorphy2 import MorphAnalyzer
import json

In [2]:
def lemmatisation(text):
    morph = MorphAnalyzer()
    stop_words = stopwords.words('russian')
    exclude = set(punctuation + '0123456789'+'–—'+'«»')
    text = ''.join(ch for ch in text if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(text.lower()) 
    tokens = [i.strip() for i in tokens if i not in stop_words]
    lemmas = [morph.parse(i)[0].normal_form for i in tokens]
    return lemmas

In [3]:
inv_index = defaultdict(set)  # словарь ключ-лемма, значение - список кортежей, состоящих из ссылки и fq
url_info = defaultdict(list)   # словарь ключ-ссылка на статью, значение - лист, состоящий из длины статьи и заголовка
#num_words = 0
#N = 0

for root, dirs, files in os.walk('./corpus_susnov/'):
    for fname in files:
        if fname[-4:] == '.txt':
            with open(root+'/'+fname, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                url = lines[4].lstrip('@url').strip()
                title = lines[1].lstrip('@ti').strip()
                lemmas = lemmatisation(' '.join(lines[5:]))
                url_info[url] = [len(lemmas), title]
                #num_words += len(lemmas)
                #N += 1
                for lemma in lemmas:
                    fq = lemmas.count(lemma)
                    inv_index[lemma].add((url, fq))
                
#avgdl = num_words/N    # средняя длина документа в коллекции

In [4]:
for key in inv_index.keys():
    inv_index[key] = list(inv_index[key])

In [5]:
with open('inv_index.json', 'w', encoding='utf-8-sig') as f:
    f.write(json.dumps(inv_index, ensure_ascii=False))
    
with open('url_info.json', 'w', encoding='utf-8-sig') as f:
    f.write(json.dumps(url_info, ensure_ascii=False))